In [2]:
import numpy as np;
import sys;
import os;
import glob;
import random;
import wavio;
import copy;
from random import shuffle;

In [3]:
import soundfile as sf

In [4]:
sys.path.append("../../../Libs/")
sys.path.append("../../../src/SharedLibs/")

In [5]:
from SharedLibs import getFileList

### Shared Data

In [37]:
sampling_rate = 20000;
desired_wav_len = 30225;#int(np.ceil(1.5 * sampling_rate));
clear_clip_file = "../../../datasets/CurrentUse/mixed_sounds/clear_clips/clear_clip.wav" 
one_half_sec_wav_name_fmt = "{}_{}.wav";
src_eng_help_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s1_processed_clear_clips/";
head_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/";
middle_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/";
tail_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/tail/";
clear_eng_help_wav_list = getFileList(src_eng_help_dir);
clear_clip = wavio.read(clear_clip_file).data.T[0];
len_of_clear_clip = len(clear_clip);

In [38]:
# print(os.path.basename(raw_eng_help_wav_list[0])[:-4])
# print(len(clear_clip))
# test_data = wavio.read(raw_eng_help_wav_list[0]).data.T[0];
# print(f"len of test_data:{len(test_data)}, shape:{test_data.shape}");
# res_data = np.append(test_data, clear_clip);
# print(f"len of res_data:{len(res_data)}, shape:{res_data.shape}");
# print(f"count of raw english help wav files:{len(raw_eng_help_wav_list)}");
# print(len([i for i in clear_clip]))
# for i in raw_eng_help_wav_list:
#     print(i)

## putting speech signal in the head and extending clip to 1.5 Sec

In [42]:
def extend_at_head_main(suffix_word="head", upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name)
        # print(save_full_name)
        if len_data < desired_wav_len:
            result = np.append(tmp_data, clear_clip[:desired_wav_len-len_data]);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [43]:
# extend_at_head_main(upperlel_save_dir=head_clip_save_dir);

1:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/man_clear_clip_help_001_71_head.wav
2:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/man_clear_clip_help_043_71_head.wav
3:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/man_clear_clip_help_014_71_head.wav
4:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/man_clear_clip_help_052_71_head.wav
5:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/man_clear_clip_help_040_71_head.wav
6:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/man_clear_clip_help_044_71_head.wav
7:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5se

## putting speech signal in the middle and extending clip to 1.5 Sec

In [33]:
def insert_middle_logic(wanted_len=None, src_wav_len=None):
    dif_len_pos = int(wanted_len - src_wav_len);
    half_dif_len = int(np.ceil(dif_len_pos/2));
    start_pos = half_dif_len;
    end_pos = start_pos + src_wav_len;
    return start_pos, end_pos

In [40]:
def extend_at_middle_main(suffix_word="middle",upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name);
        # print(save_full_name)
        if len_data < desired_wav_len:
            s_pos, e_pos = insert_middle_logic(wanted_len=desired_wav_len, src_wav_len=len_data);
            result = clear_clip[:s_pos].copy();
            result = np.append(result, tmp_data[:e_pos])
            if e_pos < desired_wav_len:
                result = np.append(result, clear_clip[:desired_wav_len-e_pos]);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [41]:
extend_at_middle_main(upperlel_save_dir=middle_clip_save_dir)

1:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/man_clear_clip_help_001_71_middle.wav
2:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/man_clear_clip_help_043_71_middle.wav
3:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/man_clear_clip_help_014_71_middle.wav
4:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/man_clear_clip_help_052_71_middle.wav
5:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/man_clear_clip_help_040_71_middle.wav
6:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/man_clear_clip_help_044_71_middle.wav
7:writing wav file to ../../CurrentUse/wav_files/src_wavs/positive/help/help_engli

## putting speech signal in the tail and extending clip to 1.5 Sec

## mix with white noises